## Referencias

* https://www.ufrgs.br/wiki-r/index.php?title=Data_Frame

In [1]:
# Adicionando o diretorio atual
setwd("/home/cipher/Desktop/EACH/MQA/MetodosQuantitativos");

dataset <- read.csv(file="./summer-products-with-rating-and-performance_2020-08.csv", header=TRUE, sep=",")
dataset <- as.data.frame(dataset)